<a href="https://colab.research.google.com/github/Keerthana8888/Covid19_CNN/blob/main/Code%20File/Covid19_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Approach To The Problem

1. Load the Images and Libaries
2. Vizulaize the Images
3. Declare a sequential model
4. Declare Layers
  * Convolutions ( Filters , Stride , Shape)
  * Pooling (Max/Avg , Stride , Shape)
  * Flatten
  * Dense Layer ( Number of neurons , Activation)
  * Output Layer (Activation , Output Size)
5. Compile the Model ( Optimizer , Loss , Metrics)
6. Fit the model (Dataset , Epochs , Batch Size)
7. Model Evaluation

#Load The Dataset And Libraries

In [ ]:
!unzip'/content/Covid 19 Image Dataset.zip'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
train_dir = "/content/covid19-image-dataset/Covid19-dataset/train"
val_dir = "/content/covid19-image-dataset/Covid19-dataset/test"

# Load The Images

In [ ]:
train_data = image_dataset_from_directory(train_dir , image_size = (160,160) ,
                                             class_names = ['Covid', 'Normal', 'Viral Pneumonia'], batch_size =64 , seed = 123)
val_data = image_dataset_from_directory(val_dir , image_size = (160,160) ,
                                           class_names = ['Covid', 'Normal', 'Viral Pneumonia'], batch_size = 64 , seed = 123)

In [ ]:
train_dataset.class_names

#Visualization Of Images

In [ ]:
clas_names = train_dataset.class_names

plt.figure(figsize = (10,10))
for images , labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype('uint8'))
    plt.title(clas_names[labels[i]])

# Declare A Sequential Model

In [ ]:
model = Sequential()

### Feature Extraction Layer ###
## Block 1 ##
model.add(Conv2D(128 , (3,3) , padding = 'same' , activation = 'relu' , input_shape = (160,160,3)))
model.add(MaxPooling2D(pool_size = (2,2) , strides = (2,2)))

## Block 2 ##
model.add(Conv2D(512 , (3,3) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2) , strides = (2,2)))

## Block 3 ##
model.add(Conv2D(1024 , (3,3) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2) , strides = (2,2)))

### Classification Layer ###
model.add(Flatten())
model.add(Dense(128 , activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(3 , activation = 'softmax'))

In [ ]:
model.summary()

#Compile The Model

In [ ]:
model.compile(optimizer = 'adam' , loss = 'sparse_classification_crossentropy' , metrics = ['accuracy'])

#Fit The Model

In [ ]:
hist = model.fit(train_dataset , epochs = 50 , validation_data = val_dataset , batch_size = 32)

#Model Evaluation

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train' , 'Validation'] , loc = 'upper left')
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train' , 'Validation'] , loc = 'upper left')
plt.show()

#Plotting Confusion Matrix

In [ ]:
y_hat_train = model.predict(train_dataset)
y_hat_test = model.predict(val_dataset)

In [ ]:
y_true_train = np.array([label.numpy() for _, labels in train_dataset for label in labels])

In [ ]:
y_true_val = np.array([label.numpy() for _, labels in val_dataset for label in labels])

In [ ]:
def model_eval(actual , predicted):
  conf_matrix = confusion_matrix(actual , predicted)
  clas_rep = classification_report(actual , predicted)
  acc_score = accuracy_score(actual , predicted)

  print("The Accuracy of the Model is:" , round(acc_score , 2))
  print(conf_matrix)
  print(clas_rep)

In [ ]:
model_eval(y_true_train,y_hat_train.round())

In [ ]:
model_eval(y_true_val,y_hat_test.round())